### __Group 9: Mazarine Tulp, Floor Halkes, and Kyra Jongman__

07-11-2024

### __Import Packages__

In [6]:
%load_ext autoreload
%autoreload 2

In [13]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

In [15]:
import os
print(os.getcwd())


/Users/Mazarinetulp/Documents/nlpass2/NLP_group_9/NLP/nlp


In [16]:
from preprocessing_functions import *


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/Mazarinetulp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Mazarinetulp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Mazarinetulp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### __Load Data__

In [9]:
# OPEN CSV FILES

# This dataset includes news and opinion articles about various companies
df = pd.read_csv('data/raw/us_equities_news_dataset.csv')

# DROP DUPLICATES

df_filtered = df.drop_duplicates(subset=['content'])

# DROP ROWS WHERE 'CONTENT' columns has NANS

df_filtered = df_filtered.dropna(subset=['content'])

# FILTER DATAFRAME ON CONTENT OF EITHER 'NVDA' OR 'NVIDIA'

# Check if each row in 'content' column contains the words "nvda" or "nvidia
df_filtered_on_content = df_filtered[df_filtered['content'].fillna('').str.contains(
    r'\bnvda\b|\bnvidia\b', case=False, regex=True)]
print(f"The amount of articles is (both news and opinions): {len(df_filtered_on_content)}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/us_equities_news_dataset.csv'

In [5]:
df_filtered_on_content.head(5)

,id,ticker,title,category,content,release_date,provider,url,article_id
24,221539,NIO,A Central Bank War Just Started And Its Good F...,opinion,ECB Effects\nThe move in the euro was huge fa...,2019-03-07,Michael Kramer,https://www.investing.com/analysis/a-central-b...,200395687
32,221547,NIO,6 Stocks To Watch Nivida Could Be Falling,opinion,6 Stocks To Watch March 6 Trading Session\nSt...,2019-03-06,Michael Kramer,https://www.investing.com/analysis/6-stocks-to...,200394931
57,221572,NIO,Stocks Dow Drops Nearly 400 Points as Apple ...,news,Investing com A rout in Apple and Facebook ...,2018-11-19,Investing.com,https://www.investing.com/news/stock-market-ne...,1694042
78,221593,UBER,The Zacks Analyst Blog Highlights Advanced Mi...,opinion,For Immediate ReleaseChicago IL January 13 ...,2020-01-12,Zacks Investment Research,https://www.investing.com/analysis/the-zacks-a...,200498277
82,221597,UBER,The Best Of CES 2020 Revised,opinion,With 4 500 companies bringing their innovation...,2020-01-16,Zacks Investment Research,https://www.investing.com/analysis/the-best-of...,200499164


### __Preprocessing Steps:__

In [6]:
# PREPROCESSING CONTENT COLUMN

# Convert to lowercase
df_filtered_on_content.loc[:,'preprocessed_content'] = convert_to_lowercase(df_filtered_on_content['content'])

/tmp/ipykernel_10503/3072511301.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_on_content.loc[:,'preprocessed_content'] = convert_to_lowercase(df_filtered_on_content['content'])


In [7]:
# Remove URLs
df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_all_urls(df_filtered_on_content['preprocessed_content'])

In [8]:
# Remove all words that are note characters
df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_non_words_characters(df_filtered_on_content['preprocessed_content'])

In [9]:
# Remove digits
df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_digits(df_filtered_on_content['preprocessed_content'])

In [10]:
# Tokenize words
df_filtered_on_content.loc[:, 'preprocessed_content'] = tokenize_words(df_filtered_on_content['preprocessed_content'])

In [11]:
# Remove stopwords
df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_stopwords(df_filtered_on_content['preprocessed_content'])

In [12]:
# Lemmatize words 
df_filtered_on_content.loc[:, 'preprocessed_content'] = lemmatize(df_filtered_on_content['preprocessed_content'])

In [28]:
# Lemmatize words - dit is de spacy versie die ik nog niet af heb gekregen 
# df_filtered_on_content.loc[:, 'preprocessed_content'] = lemmatize_words(df_filtered_on_content['preprocessed_content'])

In [14]:
# ik had het even in losse cellen gezet om te testen maar kan ook weer zo in deze cel bij elkaar

# # PREPROCESSING CONTENT COLUMN

# # Convert to lowercase
# df_filtered_on_content.loc[:, 'preprocessed_content'] = convert_to_lowercase(df_filtered_on_content['content'])

# # Remove URLs
# df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_all_urls(df_filtered_on_content['preprocessed_content'])

# # Remove all words that are note characters
# df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_non_words_characters(df_filtered_on_content['preprocessed_content'])

# # Remove digits
# df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_digits(df_filtered_on_content['preprocessed_content'])

# # Tokenize words
# df_filtered_on_content.loc[:, 'preprocessed_content'] = tokenize_words(df_filtered_on_content['preprocessed_content'])

# # Remove stopwords
# df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_stopwords(df_filtered_on_content['preprocessed_content'])

# # Lemmatize words
# df_filtered_on_content.loc[:, 'preprocessed_content'] = lemmatize_words(df_filtered_on_content['preprocessed_content'])

# df_filtered_on_content.head(5)

### __Remove least used and most used words:__

In [15]:
total_number_of_words = len(get_word_frequency(df_filtered_on_content['preprocessed_content']))
print(f"Total number of words in corpus is {total_number_of_words}")

Total number of words in corpus is 29182


In [16]:
word_frequency = get_word_frequency(df_filtered_on_content['preprocessed_content'])
word_frequency

[('year', 16037),
 ('company', 15428),
 ('stock', 15427),
 ('nasdaq', 14843),
 ('zacks', 11194),
 ('market', 10464),
 ('earnings', 9880),
 ('nvidia', 8932),
 ('quarter', 8677),
 ('share', 8199),
 ('growth', 7335),
 ('revenue', 7185),
 ('billion', 6275),
 ('also', 6241),
 ('nyse', 6181),
 ('technology', 5549),
 ('estimate', 5427),
 ('new', 5067),
 ('investor', 5027),
 ('million', 4799),
 ('rank', 4798),
 ('last', 4746),
 ('inc', 4600),
 ('week', 4462),
 ('expected', 4268),
 ('u', 4189),
 ('day', 4120),
 ('price', 4118),
 ('p', 4059),
 ('report', 4027),
 ('nvda', 3958),
 ('buy', 3929),
 ('one', 3892),
 ('investment', 3891),
 ('time', 3877),
 ('trade', 3869),
 ('data', 3785),
 ('high', 3780),
 ('per', 3637),
 ('strong', 3628),
 ('industry', 3606),
 ('like', 3551),
 ('index', 3451),
 ('chip', 3368),
 ('top', 3361),
 ('consensus', 3235),
 ('see', 3232),
 ('intel', 3222),
 ('analyst', 3205),
 ('sector', 3179),
 ('product', 3179),
 ('semiconductor', 3132),
 ('business', 3087),
 ('china', 3081

In [17]:
most_common_words = word_frequency[:20]
print(f"20 most common words are \n {most_common_words}")

20 most common words are 
 [('year', 16037), ('company', 15428), ('stock', 15427), ('nasdaq', 14843), ('zacks', 11194), ('market', 10464), ('earnings', 9880), ('nvidia', 8932), ('quarter', 8677), ('share', 8199), ('growth', 7335), ('revenue', 7185), ('billion', 6275), ('also', 6241), ('nyse', 6181), ('technology', 5549), ('estimate', 5427), ('new', 5067), ('investor', 5027), ('million', 4799)]


In [18]:
get_number_of_words_with_up_to_n_times_present(word_frequency, 5)

19223

In [19]:
# look at which words are used n times 
[word for word in word_frequency if word[1] == 5]

[('bunds', 5),
 ('deflate', 5),
 ('refuge', 5),
 ('folding', 5),
 ('thinkpad', 5),
 ('cube', 5),
 ('brush', 5),
 ('neon', 5),
 ('powervision', 5),
 ('aerial', 5),
 ('avoidance', 5),
 ('debuting', 5),
 ('attaching', 5),
 ('achievable', 5),
 ('sdks', 5),
 ('localization', 5),
 ('impractical', 5),
 ('awful', 5),
 ('weathered', 5),
 ('tired', 5),
 ('accompanying', 5),
 ('timetable', 5),
 ('meli', 5),
 ('tourist', 5),
 ('humanoid', 5),
 ('ridesharing', 5),
 ('commuter', 5),
 ('cybertruck', 5),
 ('prep', 5),
 ('ingest', 5),
 ('hook', 5),
 ('pocketed', 5),
 ('neumann', 5),
 ('plowing', 5),
 ('fisv', 5),
 ('upping', 5),
 ('paced', 5),
 ('ebbed', 5),
 ('noisy', 5),
 ('mojo', 5),
 ('trepidation', 5),
 ('mightily', 5),
 ('questionnaire', 5),
 ('reserved', 5),
 ('defective', 5),
 ('caesar', 5),
 ('bidder', 5),
 ('woo', 5),
 ('celebrity', 5),
 ('coupon', 5),
 ('academy', 5),
 ('clearance', 5),
 ('punitive', 5),
 ('chipped', 5),
 ('outdone', 5),
 ('ix', 5),
 ('nussey', 5),
 ('improbable', 5),
 ('bur

In [20]:
# find n most frequent words 
most_frequent_words = n_most_frequent_words(df_filtered_on_content['preprocessed_content'], 5)
most_frequent_words

['year', 'company', 'stock', 'nasdaq', 'zacks']

In [21]:
# create new column with word_removal ( ik doe dit omdat het niet altijd goed is om woorden te verwijderen, dan kunnen we nog even kijken wat beter is )
# remove n most frequent words from preprocessed content 

df_filtered_on_content.loc[:, 'most_word_removal'] = multiple_word_remove_func(df_filtered_on_content['preprocessed_content'], most_frequent_words)

/tmp/ipykernel_10503/2488872262.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_on_content.loc[:, 'most_word_removal'] = multiple_word_remove_func(df_filtered_on_content['preprocessed_content'], most_frequent_words)


In [22]:
least_used = n_times_least_used(df_filtered_on_content['preprocessed_content'], 5)      # we moeten even bepalen hoeveel we willen verwijderen 

In [23]:
df_filtered_on_content.loc[:, 'least_word_removal'] = multiple_word_remove_func(df_filtered_on_content['preprocessed_content'], least_used)

/tmp/ipykernel_10503/4246337592.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_on_content.loc[:, 'least_word_removal'] = multiple_word_remove_func(df_filtered_on_content['preprocessed_content'], least_used)


In [24]:
df_filtered_on_content.head()

,id,ticker,title,category,content,release_date,provider,url,article_id,preprocessed_content,most_word_removal,least_word_removal
24,221539,NIO,A Central Bank War Just Started And Its Good F...,opinion,ECB Effects\nThe move in the euro was huge fa...,2019-03-07,Michael Kramer,https://www.investing.com/analysis/a-central-b...,200395687,"[ecb, effect, move, euro, huge, falling, pip, ...","[ecb, effect, move, euro, huge, falling, pip, ...","[ecb, effect, move, euro, huge, falling, pip, ..."
32,221547,NIO,6 Stocks To Watch Nivida Could Be Falling,opinion,6 Stocks To Watch March 6 Trading Session\nSt...,2019-03-06,Michael Kramer,https://www.investing.com/analysis/6-stocks-to...,200394931,"[stock, watch, march, trading, session, stock,...","[watch, march, trading, session, went, pretty,...","[stock, watch, march, trading, session, stock,..."
57,221572,NIO,Stocks Dow Drops Nearly 400 Points as Apple ...,news,Investing com A rout in Apple and Facebook ...,2018-11-19,Investing.com,https://www.investing.com/news/stock-market-ne...,1694042,"[rout, apple, facebook, nasdaq, fb, monday, sw...","[rout, apple, facebook, fb, monday, swept, tec...","[rout, apple, facebook, nasdaq, fb, monday, sw..."
78,221593,UBER,The Zacks Analyst Blog Highlights Advanced Mi...,opinion,For Immediate ReleaseChicago IL January 13 ...,2020-01-12,Zacks Investment Research,https://www.investing.com/analysis/the-zacks-a...,200498277,"[immediate, releasechicago, il, january, annou...","[immediate, releasechicago, il, january, annou...","[immediate, releasechicago, il, january, annou..."
82,221597,UBER,The Best Of CES 2020 Revised,opinion,With 4 500 companies bringing their innovation...,2020-01-16,Zacks Investment Research,https://www.investing.com/analysis/the-best-of...,200499164,"[company, bringing, innovation, ce, jan, get, ...","[bringing, innovation, ce, jan, get, really, d...","[company, bringing, innovation, ce, jan, get, ..."


### __Save preprocessed data to parquet__

In [36]:
df_filtered_on_content.to_parquet('data/processed/preprocessed_df.parquet')